## Regime Transiente 2D Difusão de Calor

Equação 2D:

$$\frac{\partial^2 T}{\partial^2 x}  + \frac{\partial^2 T}{\partial^2 y} = \frac{1}{\alpha} \frac{\partial T}{\partial t}  \tag{1}$$

Onde difusividade térmica é:

$$ \alpha = \frac{k}{\rho c} $$ em $ \frac{m^2}{s} $

Em um grid regular, utilizando discretização em diferenças finitas usando diferenças centradas para o espaço:

O indice de tempo é **n** e indice do eixo x é **i** e no eixo y é **j**.  

$$
\frac{\partial^2 T^n_{ij}}{\partial^2 x}  = \frac{T_{i+1j}^n-2T_{ij}^n+T_{i-1j}^n}{\Delta x^2}  
\tag{2}
$$

$$
\frac{\partial^2 T^n_{ij}}{\partial^2 y}  = \frac{T_{ij+1}^n-2T_{ij}^n+T_{ij-1}^n}{\Delta y^2}  
\tag{3}
$$

e diferença progressiva para o tempo nos indices de tempo **n** e **n+1** ("futuro"):

$$ \frac{\partial T_{ij}^n}{\partial t} = \frac{T_{ij}^{n+1}-T_{ij}^n}{\Delta t} \tag{4} $$






Combinando as equações (2), (3) e (4) chegamos em um esquema de "marcha" em (5).  
Onde os valores **n+1** ("futuro") podem ser calculados a partir dos valores em **n**: 

$$ \frac{T_{ij}^{n+1}-T_{ij}^n}{\Delta t} \frac{1}{\alpha} = \frac{T_{i+1j}^n-2T_{ij}^n+T_{i-1j}^n}{\Delta x^2} + \frac{T_{ij+1}^n-2T_{ij}^n+T_{ij-1}^n}{\Delta y^2}$$


Considerando $\Delta x = \Delta y = \Delta s$

$$ T_{ij}^{n+1} =   \frac{\alpha \Delta t}{\Delta s^2}\left(T_{i+1j}^n-2T_{ij}^n+T_{i-1j}^n+T_{ij+1}^n-2T_{ij}^n+T_{ij-1}^n\right)  + T_{ij}^n $$

$$ T_{ij}^{n+1} = \lambda \left(T_{i+1j}^n+T_{i-1j}^n+T_{ij+1}^n+T_{ij-1}^n-4T_{ij}^n\right)  + T_{ij}^n \tag{5}$$

onde:

$$ \lambda = \frac{\alpha \Delta t}{\Delta s^2} \tag{6} $$ 

Para garantir estabilidade numérica do esquema conforme orientação $\lambda \leq \frac{1}{2}$

Para garantir estabilidade numérica para 2D $\Delta t \leq \frac{ \Delta x^2}{4} $


** Difusividade alumínio 97 mm²/s = 0.97 cm²/s **

### Condições de contorno

Esquerda e topo 100 graus  
Direita e base 50 graus  

In [1]:
from numba import njit, prange

@njit(parallel=True)
def passoDifusao(tp1, tp, lmbda, ns):
    for i in prange(1, ns-1):
        for j in range(1, ns-1):
            tp1[j, i] = lmbda*(tp[j, i+1]+tp[j, i-1]+tp[j+1, i]+tp[j-1, i]-4*tp[j, i]) + tp[i, j]           
    

In [2]:
import numpy as np
from scipy.signal import fftconvolve

def rodarSimulacao(N, duracao, every=10, lmbda=0.04175, dt=None):
    """
    * N : numero de pontos discretização em x e y
    * duracao : em segundos da simulacao da difusao
    * lmbda : lambda predefido para garantir estabilidade numérica 
    igual à 0.04175 
    * every : numero de 'snapshots' para salvar  
    
    O numero de iteracoes é calculado baseado na duracao
    """
    alpha = 0.97 # alpha alumínio 
    ds = 10./(N-1) # 10 cm / N-1 pontos discretização espaço 
    if dt is None: # nao foi especificado
        dt = 0.7*(ds**2/4.) # garantindo dt < dx^2/4
    lmbda = alpha*dt/(ds**2) # calcula novamente
    # "paineis" com temperatura da placa em tn e tn+1 (derivada primeira 1 painel necessario)
    tp1 = np.zeros((N, N)) # temperatura da placa em tn+1 
    tp = np.zeros((N, N)) # temperatura da placa em tn 
    
    # condições de contorno iniciais no tempo infinito 
    # extremidades da placa 
    tp[:,  0] = 100. # esquerda 
    tp[0,  :] = 100. # acima
    tp[:, -1] = 50. # direita    
    tp[-1,  :] = 50. # abaixo    
    tp1[:, :] = tp[:, :]    
    niter = int(duracao/dt) # numero de iteracoes necessarias para essa duracao
    nsave = int(niter/every) # step de iteracoes para salvar 'snapshot'
    snapshots = np.zeros((nsave+1, N, N)) # 'filme' das iteracoes
    isnap = 0 # indice para salvar 'snapshots' das iteracoes        
    
    for n in range(niter): # marcha
        # calcula tn+1 em funcao de f(tn) 
        passoDifusao(tp1, tp, lmbda, N)
        tp[:,:] = tp1[:,:] # atualiza pilha
        if n%every==0:
            snapshots[isnap] = tp
            isnap += 1 # p/ próximo 'frame' do 'filme'

    # imprime um resumo de alguns parâmetros
    print('Dt: {:.3g} numero de Iteracoes: {:5d} '.format(dt, niter))
    print('lambda : ', lmbda)
    
    # salva o último
    snapshots[-1] = tp
    # retorna o 'filme' de snapshots, e o intervalo em x    
    return snapshots, ds

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt

### Para um grid com 6 pontos em cada lado

In [4]:
ns = 6

In [5]:
%%time
snapshots, dx = rodarSimulacao(ns, 25., every=10)

Dt: 0.7 numero de Iteracoes:    35 
lambda :  0.16974999999999998
CPU times: user 424 ms, sys: 4 ms, total: 428 ms
Wall time: 428 ms


In [6]:
snaps = np.array(snapshots)

In [7]:
import matplotlib
from matplotlib import animation

ext = [0, ns*dx, ns*dx, 0]
fig = plt.figure(figsize=(6,6))
im = plt.imshow(snaps[0, :, :], vmin=0, vmax=100, 
                extent=ext, alpha=0.7)
plt.colorbar(im,  shrink=0.6, pad = 0.01, aspect=10, label=u'temperatura')
plt.xlabel('x (cm)')
plt.ylabel('z (cm)')
plt.tight_layout()
plt.close()

def animate(i):
    im.set_data(snaps[i, :,:])

ani = animation.FuncAnimation(fig, animate, frames=int(snaps.shape[0]))

from IPython.display import HTML
HTML(ani.to_jshtml())

_________________________________________________________

### Para um grid com 21 pontos em cada lado

In [12]:
ns = 21

In [13]:
%%time
snapshots, dx = rodarSimulacao(ns, 2., every=100)

Dt: 0.0437 numero de Iteracoes:    45 
lambda :  0.16974999999999998
CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 2.91 ms


In [14]:
snaps = np.array(snapshots)

In [15]:
import matplotlib
from matplotlib import animation

ext = [0, ns*dx, ns*dx, 0]
fig = plt.figure(figsize=(6,6))
im = plt.imshow(snaps[0, :, :], vmin=0, vmax=100, extent=ext, alpha=0.7)
plt.colorbar(im,  shrink=0.6, pad = 0.01, aspect=10, label=u'temperatura')
plt.xlabel('x (cm)')
plt.ylabel('z (cm)')
plt.tight_layout()
plt.close()

def animate(i):
    im.set_data(snaps[i, :,:])

ani = animation.FuncAnimation(fig, animate, frames=int(snaps.shape[0]))

from IPython.display import HTML
HTML(ani.to_jshtml())

# 3.
Compare os erros das solucoes numericas de equilıbrio (nos 2 casos anteriores) em relacao a solu¸cao analıtica  
(uma reta que passa pelos pontos de temperatura 100◦e 50◦C nos extremos), em especial no caso de utilizacao  
de λ = 1/6 (que tende a minimizar os erros);  

A solução analítica pode ser escrita como uma reta:

$$ f(x) = 100 -5x $$ 

Lembrando que a barra tem 10 cm e logo que $\Delta x = \frac{10}{N-1} $.  
Escrevendo em função dos **N** pontos de discretização e $ \Delta x $


$$ f(i\Delta x) = 100 -5(i\Delta x) $$
$$ f(i) = 100 -5(i\Delta x) $$

Onde $i$ represente um indíce de posição na barra.